# Importandos bibliotecas

In [7]:
import csv
import pandas as pd
from bs4 import BeautifulSoup
from msedge.selenium_tools import Edge, EdgeOptions

# Scraper

In [4]:
def get_url(pesquisa):
    url = f'https://www.amazon.com.br/s?k={pesquisa}&i=computers&__mk_pt_BR=ÅMÅŽÕÑ&crid=3VN7CQJP4L6O2&sprefix=notebo%2Ccomputers%2C301&ref=nb_sb_noss_2'
    pesquisa = pesquisa.replace(' ','+')
        
    # adicionando pagina para url
    url += '&page={}'
    
    return url

def extraindo_dados(item):
    
    card = {}
    
    # adicionando descrição
    atag = item.h2.a
    card['Descrição'] = atag.text.strip()
    
    try:
        # adicionando preco
        prince_parent = item.find('span','a-price')
        card['Preço'] = "".join(prince_parent.find('span','a-offscreen').text.split())
    except AttributeError:
        return 
    
    try:
        # adicionando estrlas e total de reviews
        card['Estrelas'] = item.i.text
        card['TotalReviews'] = item.find('span',{'class':'a-size-base'}).text
    except AttributeError:
        card['Estrelas'] = ''
        card['TotalReviews'] = ''

    card['Url'] = 'https://www.amazon.com.br'+atag.get('href')

    return card

def main(pesquisa):
    
    # Ativando webdriver
    options = EdgeOptions()
    options.use_chromium = True
    driver = Edge(options=options)
    
    cards = []
    url = get_url(pesquisa)

    # Obtendo total de paginas
    driver.get(url)
    soup_sl = BeautifulSoup(driver.page_source, 'html.parser')
    list_pages = soup_sl.find('ul',{'class':'a-pagination'}).findAll('li')
    list_pages.pop()
    list_pages.pop(0)
    total_pages = int(list_pages[-1].getText())
    print(f'total de páginas = {total_pages}')
    
    for page in range(total_pages):
        driver.get(url.format(page+1))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div',{'data-component-type':'s-search-result'})
        
        for item in results:
            card = extraindo_dados(item)
            if card:
                cards.append(card)
                
    driver.close()
    
    #Salvando dados em csv com pandas
    dataset = pd.DataFrame(cards)
    dataset.to_csv(f'output/dataset_{pesquisa}.csv'.replace(' ','_'), index = False, encoding = 'utf-8-sig')
    dataset

In [5]:
main('smartphone')

total de páginas = 7


# Observando dados

In [6]:
pd.read_csv('output/dataset_smartphone.csv')

,Descrição,Preço,Estrelas,TotalReviews,Url
0,Estabilizador de Imagem Zhiyun Smooth Q3 para ...,"R$587,00","4,6 de 5 estrelas",6.0,https://www.amazon.com.br/gp/slredirect/picass...
1,"Tripé Para Celular e Camera, Tripé Profissiona...","R$79,90","5,0 de 5 estrelas",5.0,https://www.amazon.com.br/gp/slredirect/picass...
2,Domary Mini teclado virtual de toque portátil ...,"R$445,67",NaN,NaN,https://www.amazon.com.br/gp/slredirect/picass...
3,Daerzy Phone Octopus Tripod 1.5KG flexível e a...,"R$134,99",NaN,NaN,https://www.amazon.com.br/gp/slredirect/picass...
4,"Smartphone Positivo Q20, 4Gb Ram, 128Gb, Tela ...","R$799,00","4,1 de 5 estrelas",832.0,https://www.amazon.com.br/Smartphone-Positivo-...
...,...,...,...,...,...
239,"Impressora de etiquetas Bluetooth portátil, vá...","R$199,00",NaN,NaN,https://www.amazon.com.br/Impressora-dispon%C3...
240,Suporte de Mesa Universal para Celular Tablet ...,"R$11,99","4,1 de 5 estrelas",88.0,https://www.amazon.com.br/Suporte-Universal-Ce...
241,Yongluo Obturador de câmera Bluetooth obturado...,"R$51,19",NaN,NaN,https://www.amazon.com.br/gp/slredirect/picass...
242,Yongluo F6 Smartphone Gimbal 3-Axis Handheld S...,"R$628,09",NaN,NaN,https://www.amazon.com.br/gp/slredirect/picass...
